# Notes

This notebook tests codes which loop over pipes created for WHIM analysis by make_pipes_5.py and corrects their placeholder subhalo ID values with true subhalo ID values.

It adapts code from:
- Pipe_LSS_Analysis_5.ipynb
- make_pipes_5.py

# Imports

In [1]:
import os
import random

import numpy as np
import operator as op
import illustris_python as il

from frb.dm import igm
from charlie_TNG_tools import temp2u
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import binned_statistic_dd

from astropy import units as u
from numpy import random as rand
from astropy import constants as c
from matplotlib import pyplot as plt
from astropy.cosmology import Planck15 as cosmosource
from astropy.modeling import models, fitting



# Functions

In [2]:
#method for histogramming over one axis from https://stackoverflow.com/questions/40018125/binning-of-data-along-one-axis-in-numpy

def hist_1d(data,bins=np.linspace(0,5000,100),density=True):
    """
    histograms data along one axis
    adapted from https://stackoverflow.com/questions/40018125/binning-of-data-along-one-axis-in-numpy
    
    INPUTS:
    
    data : [arr]  array of data to histogram
    density : [boolean] True or False: whether the histogram should normalise for the pdf or not.
    """
    
    return np.histogram(data,bins=bins,density=density)[0]

In [3]:
#function to darken colours in plots from: https://stackoverflow.com/questions/37765197/darken-or-lighten-a-color-in-matplotlib

def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])

In [4]:
#function to get true subhalo id information (from make_pipes_5.py)

def newpIDshIDconverter(pipe_cell_pIDs,ChunkedPartIDs,ChunkedSubhIDs):
    """
    New version of the code which creates a set of subhalo IDs from a set of particle
    IDs.
    
    This version loops through each chunk of the simulation ID lists in turn searching
    for relevant particle and subhalo IDs. 
    
    Note: could be improved to be faster if, when all correct particle IDs are found, it
    does not need to search further chunks. This is not yet implemented.
    
    INPUTS:
    
    pipe_cell_pIDs : the particle ids of cells in a given pipe
    ChunkedPartIDs : list containing locations of the chunks of 
                     the particle ID list for all cells in the 
                     desired simulation. If all of these were loaded
                     into a single array, the result would be the
                     same as AllPartIDs in oldpIDshIDconverter().
    ChunkedSubhIDs : list containing locations of the chunks of 
                     the subhalo ID list for all cells in the 
                     desired simulation. If all of these were loaded
                     into a single array, the result would be the
                     same as AllPartIDs in oldpIDshIDconverter().
    
    RETURNS:
    
    pipe_cell_shIDs : the corresponding subhalo IDs for every particle IDs.

    
    """
    
    #create a set of particle IDs for the cells in this pipe
    PartID_Set = set(pipe_cell_pIDs.tolist())
    
    #initialise an array to contain all subhalo IDs in simulation
    pipe_cell_shIDs = np.ones(pipe_cell_pIDs.shape,dtype=int)*-1
    
    #print(ChunkedPartIDs)
    
    #load chunks of the all-simulation particle ID list
    for i in range(len(ChunkedPartIDs)):
        
        #True/False array of same shape as pipe data which allows 
        #us to extract the relevant IDs from each chunk
        TF_arr = np.full(pipe_cell_pIDs.shape, False) #begin with false, flip to true when chunk contains ID
        
        sim_inds = []#initialise an array to contain  positions of any cells in this chunk
        
        #get location of chunks to load
        PartFile_toload = ChunkedPartIDs[i] #all-simulation particle ID list chunk
        SubhFile_toload = ChunkedSubhIDs[i] #all-simulation subhalo ID list chunk
        
        #load the ID list chunks
        ChunkOfPartIDs = np.load(PartFile_toload) #particle chunk
        ChunkOfSubhIDs = np.load(SubhFile_toload) #subhalo chunk

        #loop over the particle IDs in the chunk
        for j, x in enumerate(ChunkOfPartIDs):
            
            #find if particle ID is also in the pipe
            if x in PartID_Set:
                                
                #find where that particle ID is in the pipe
                pipe_idx = np.where(pipe_cell_pIDs==x)
                
                #flip the True/False array index to True for this cell
                TF_arr[pipe_idx] = True
                
                #append the cell's corresponding chunk position
                sim_inds.append(j)
                
                #print(i,j,pipe_idx,x,ChunkOfSubhIDs[j])

            
        #convert all chunk position indices to array
        sim_inds = np.array(sim_inds)
        
        #record all corresponding subhalo IDs in this chunk
        #print(pipe_cell_shIDs[TF_arr])
        #print(sim_inds)
        #print(ChunkOfSubhIDs[sim_inds])
        if sim_inds.size>0:#only try this if the array is not empty
            pipe_cell_shIDs[TF_arr] = ChunkOfSubhIDs[sim_inds]
            

    return pipe_cell_shIDs

# Load Pipes

In [167]:
#############################
#Identify files to be loaded#
#############################

#simulation to load data for
sim_to_load = 'TNG300-1'

#path to simulation data
basePath = '/ptmp/cwalker/Illustris_FRB_Project/TNG_copies/virgo/simulations/IllustrisTNG/{0}/output/'.format(sim_to_load)


#base directory containing pipe files for data with WHIM info
pipeBasePath = './SpeedTempTest/'

#load the files from the base path
all_files = os.listdir(pipeBasePath)#for pipes created for smaller simulations

#select all .npy files in directory
npy_files = [i for i in all_files if '.npy' in i]

#select all ddmdz (i.e. pipe) files in directory
dDMdz_files = [i for i in npy_files if 'dDMdz_Output' in i]

#select files of correct simulation
dDMdz_files = [i for i in npy_files if sim_to_load in i]

#make sure name fits the WHIM output files
dDMdz_files = [i for i in dDMdz_files if 'SpeedTempTest' in i]
        
#create a list of snapshots containing pipes in this directory
snap_list = [int(i.split('_')[3]) for i in dDMdz_files]

#sort list into ascending order (i.e. high to low redshift)
dDMdz_files = [x for _, x in sorted(zip(snap_list, dDMdz_files))]
snap_list.sort()

#reverse list into descending order (i.e. low to high redshift)
dDMdz_files = dDMdz_files[::-1]
snap_list = snap_list[::-1]

#crop to redshift 5, because that is as far as the theory goes in FRB package.
#Note that after z=3 accuracy 'is not guaranteed'

dDMdz_files = dDMdz_files[0:1] #currently just test first snapshot
snap_list = snap_list[0:1] #currently just test first snapshot

#print to check which files will be processed.
print(snap_list)
print(dDMdz_files)

[99]
['Sim_TNG300-1_Snap_99_dDMdz_Output_SpeedTempTest.npy']


In [168]:
#loop over and load files

for i in range(len(dDMdz_files)):#loop over files

    #load and extract dats
    snapshot = dDMdz_files[i].split('_')[3] #extract snapshot number
    
    print('Loading snapshot: {0}'.format(snapshot))
    
    vals = np.load(pipeBasePath+dDMdz_files[i],allow_pickle=True).tolist() #extract values from file
    
    #explore what data exists for a file
    print('Keys in these files: {0}'.format(vals.keys()))

    #explore the particle id key which will be used to create subhalo ids
    print('number of sightlines in file: {0}'.format(len(np.array(vals['dDMdz_Pakmor']))))
    print('number of Particle ID lists in file: {0}'.format(len(np.array(vals['LoSPartIDs']))))

Loading snapshot: 99
Keys in these files: dict_keys(['dDMdz_Zhang', 'dDMdzHalo_Zhang', 'dDMdzFilament_Zhang', 'dDMdzVoid_Zhang', 'nHalo_Zhang', 'nFilament_Zhang', 'nVoid_Zhang', 'dDMdz_Pakmor', 'dDMdzHalo_Pakmor', 'dDMdzFilament_Pakmor', 'dDMdzVoid_Pakmor', 'nHalo_Pakmor', 'nFilament_Pakmor', 'nVoid_Pakmor', 'firstShID', 'uniqueShIDs', 'closestCoords', 'LoSBinCoords', 'LoSCellCoords', 'LoSDensities', 'LoSInternalEnergies', 'LoSElAbs', 'LoSPartIDs'])
number of sightlines in file: 5125
number of Particle ID lists in file: 5125


In [169]:
# get matchlist chunks for conversion

for snap_number in snap_list:
    print('snapshot to process: {0}'.format(snap_number))
    
    #get list of matchlist chunks
    print('Using new version of code which loads particle/subhalo matchlists in chunks from /ptmp/')
    Chunked_loc = '/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/Matchlist_dir_{0}/'.format(sim_to_load) #location of the chunked data

    #get the particle ID list chunks
    ChunkedPartIDs = os.listdir(Chunked_loc)
    ChunkedPartIDs = ['{0}/{1}'.format(Chunked_loc,i) for i in ChunkedPartIDs if 'PartList_Snap{0}_Chunk'.format(snap_number) in i]
    ChunkedPartIDs.sort()

    #get the subhalo ID list chunks
    ChunkedSubhIDs = os.listdir(Chunked_loc)
    ChunkedSubhIDs = ['{0}/{1}'.format(Chunked_loc,i) for i in ChunkedSubhIDs if 'ShIDList_Snap{0}_Chunk'.format(snap_number) in i]
    ChunkedSubhIDs.sort()
    
print(ChunkedPartIDs,ChunkedSubhIDs)


snapshot to process: 99
Using new version of code which loads particle/subhalo matchlists in chunks from /ptmp/
['/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/Matchlist_dir_TNG300-1//PartList_Snap99_Chunk0000.npy', '/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/Matchlist_dir_TNG300-1//PartList_Snap99_Chunk0001.npy', '/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/Matchlist_dir_TNG300-1//PartList_Snap99_Chunk0002.npy', '/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/Matchlist_dir_TNG300-1//PartList_Snap99_Chunk0003.npy', '/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/Matchlist_dir_TNG300-1//PartList_Snap99_Chunk0004.npy', '/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/Matchlist_dir_TNG300-1//PartList_Snap99_Chunk0005.npy', '/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/Matchlist_dir_TNG300-1//PartList_Snap99_Chunk0006.npy', '/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/Matchlist_dir_TNG300-1//PartList_Snap99_Chunk0007.npy', '/ptmp/cwalker/Illustri

# Data tests

## Checking size of chunked TNG300-1 matchlists

In [170]:
small_p = np.load('./Sim_Matchlists/Matchlist_dir_TNG50-4/PartList_Snap99.npy')
small_s = np.load('./Sim_Matchlists/Matchlist_dir_TNG50-4/ShIDList_Snap99.npy')
print(small_p.shape,small_s.shape)

med_p = np.load('/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/Matchlist_dir_TNG100-3/PartList_Snap99.npy')
med_s = np.load('/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/Matchlist_dir_TNG100-3/ShIDList_Snap99.npy')
print(med_p.shape,med_s.shape)

chunk_p = np.load('/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/Matchlist_dir_TNG300-1/PartList_Snap99_Chunk0000.npy')
chunk_s = np.load('/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/Matchlist_dir_TNG300-1/ShIDList_Snap99_Chunk0000.npy')
print(chunk_p.shape,chunk_s.shape)


(18540104,) (18540104,)
(88935326,) (88935326,)
(14454722,) (14454722,)


## Comparing chunked vs non-chunked TNG50-4 matchlists

In [171]:
#whole matchlists
p_50_whole = np.load('./Sim_Matchlists/Matchlist_dir_TNG50-4/PartList_Snap99.npy')
s_50_whole = np.load('./Sim_Matchlists/Matchlist_dir_TNG50-4/ShIDList_Snap99.npy')

#chunks of matchlists
p_50_chunk = np.load('/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/Matchlist_dir_TNG50-4/PartList_Snap99_Chunk0000.npy')
s_50_chunk = np.load('/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/Matchlist_dir_TNG50-4/ShIDList_Snap99_Chunk0000.npy')

#############################################
#compare sizes of whole matchlists vs chunks#
#############################################

#whole matchlists
print('comparing whole matchlist sizes')
print(p_50_whole.shape,s_50_whole.shape)
if p_50_whole.shape==s_50_whole.shape:
    print('Whole matchlists are of equal size')
else:
    print('Error: particle and subhalo matchlists are not of equal size')
    
#chunks
print('comparing sizes of chunks')
print(p_50_chunk.shape,s_50_chunk.shape)
if p_50_chunk.shape==s_50_chunk.shape:
    print('Chunks of matchlists are of equal size')
else:
    print('Error: chunks of matchlists are of different sizes')

    
#######################################################################################
#compare combined subhalo ID chunks to whole subhalo ID matchlist (should be the same)#
#######################################################################################

#get list of chunks
path = '/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/Matchlist_dir_TNG50-4/'
all_s_chunks = os.listdir(path)
all_s_chunks = [i for i in all_s_chunks if 'ShIDList' in i]
all_s_chunks.sort()

#combine
combined_s_chunks = np.array([])
for i in all_s_chunks:
    combined_s_chunks = np.append(combined_s_chunks,np.load(path+i))
print(combined_s_chunks.shape)
if np.all(combined_s_chunks==s_50_whole):
    print('Combined subhalo ID chunks are the same as whole matchlist')
    print('Subhalo ID chunks are not the problem.')
else:
    print('Error: combined chunks are not the same as whole matchlist')
    

#########################################################################################
#compare combined particle iD chunks to whole particle ID matchlist (should be the same)#
#########################################################################################

#get list of chunks
path = '/ptmp/cwalker/Illustris_FRB_Project/Sim_Matchlists/Matchlist_dir_TNG50-4/'
all_p_chunks = os.listdir(path)
all_p_chunks = [i for i in all_p_chunks if 'PartList' in i]
all_p_chunks.sort()

#combine
combined_p_chunks = np.array([])
for i in all_p_chunks:
    combined_p_chunks = np.append(combined_p_chunks,np.load(path+i))
print(combined_p_chunks.shape)
if np.all(combined_p_chunks==p_50_whole):
    print('Combined particle ID chunks are the same as whole matchlist')
else:
    print('Error: combined chunks are not the same as whole matchlist')
    print('Combined chunks are of size: {0}'.format(combined_p_chunks.shape))
    print('whole list is of size: {0}'.format(p_50_whole.shape))
    print('size difference is: {0}'.format(len(p_50_whole)-len(combined_p_chunks)))
    print('Particle ID chunks are the problem')
    

    
#########################################################################################
#compare whether the first chunk is the same as the beginning of the whole pID matchlist#
#########################################################################################
print('length of first pID chunk: {0}'.format(len(p_50_chunk)))
if np.all(p_50_chunk == combined_p_chunks[0:len(p_50_chunk)]):
    print('The first pID chunk is identical to the beginning of the whole matchlist')
else:
    print('Error: the first chunk is different to the beginning of the whole matchlist')
if np.all(combined_p_chunks == p_50_whole[0:len(combined_p_chunks)]):
    print('The first {0} elements of the pID chunks combined are equivalent to those elements in the whole matchlist')
else:
    print('Error: The first {0} elements of the pID chunks combined are not equivalent to those elements in the whole matchlist')



comparing whole matchlist sizes
(18540104,) (18540104,)
Whole matchlists are of equal size
comparing sizes of chunks
(18540,) (18540,)
Chunks of matchlists are of equal size
(18540104,)
Combined subhalo ID chunks are the same as whole matchlist
Subhalo ID chunks are not the problem.
(18540104,)
Combined particle ID chunks are the same as whole matchlist
length of first pID chunk: 18540
The first pID chunk is identical to the beginning of the whole matchlist
The first {0} elements of the pID chunks combined are equivalent to those elements in the whole matchlist


# simple numpy where test

In [201]:
#simple array test

chunk_p=np.arange(70) #the chunk of particle ids
chunk_s=np.arange(70)+1 #the chunk sh IDs
random.shuffle(chunk_p)
print('chunk of pIDs', chunk_p)
print('corresponding chunk of sIDs',chunk_s)

pipe_p=np.array([0,5,100,60]) #pipe particle ids #note: this method works where no particle IDs are repeated
pipe_p=np.array([0,5,100,60,5,7,5,100,6,60]) #pipe particle ids #note: method fails where particle IDs are repeated. Must be fixed.

#this can be fixed with NUMPY UNIQUE!!!!

#get unique particle IDs in pipe, and also inverse array and counts for each unique element
pipe_p_uniques,pipe_p_inverse,pipe_p_counts = np.unique(pipe_p,return_inverse=True,return_counts=True)

print('pipe pIDs',pipe_p)
print('unique pipe pIDs',pipe_p_uniques)

#initialise a unique subhalo ID list for pipe
pipe_s_uniques=np.ones_like(pipe_p_uniques)*-1 #initialise pipe shIDs
print('unique pipe sIDs: ',pipe_s_uniques)

print('Performing test')

#find which pipe pIDs are in the chunk
mask_1 = np.isin(pipe_p_uniques,chunk_p)
print('Unique pipe pIDs which are in the chunk: ',mask_1)
print('Indices in the unique pID list: ',np.argwhere(mask_1).ravel())
print('Values of these unique pIDs: ',pipe_p_uniques[mask_1])

#find which chunk IDs are in the pipe
mask_2=np.isin(chunk_p,pipe_p_uniques)
print('pIDs in the chunk which appear in the pipe: ',mask_2)
print('Indices of these in the chunk: ',np.argwhere(mask_2).ravel())
print('Values of these pIDs (in order they apper in chunk): ',chunk_p[mask_2])
print('Corresponding sIDs',chunk_s[mask_2])

#create dictionary which can convert unique pIDs to their sIDs
z = dict(zip(chunk_p[mask_2],chunk_s[mask_2]))
print('Zipped',z)

#do the conversion for the uniques
for i in range(len(pipe_p_uniques)):
    try:
        pipe_s_uniques[i]=z[pipe_p_uniques[i]]
    except:
        pass
print('unique pipe_subhalo IDs: ', pipe_s_uniques)

#recreate the non-uniques version of pIDs with the inverse array and counts
recreated_pipe_p = pipe_p_uniques[pipe_p_inverse]
pipe_s = pipe_s_uniques[pipe_p_inverse]
print('recreated pipe pIDs: {0}'.format(recreated_pipe_p))
print('associated shID vals: {0}'.format(pipe_s))

#print(pipe_p_uniques)
#print(pipe_p_inverse)
#print(pipe_p_counts)
#print(pipe_p_uniques[pipe_p_inverse])

#test2 = np.arange(4)
#np.isin([5,6],test2).any()

chunk of pIDs [61 62 20  4 56 66 40 42 68 60 13 51 21 29 36 44 35 48 26 19 47 10 41  0
 31 52  5 53 64 12 38 39 28 57 24 65 25  6 69 37 50 27 11 22 49  2 33  1
 67 15 43 45 63 32 34 54 23 30 55 14 59 17  8 16  9 46  7 18  3 58]
corresponding chunk of sIDs [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70]
pipe pIDs [  0   5 100  60   5   7   5 100   6  60]
unique pipe pIDs [  0   5   6   7  60 100]
unique pipe sIDs:  [-1 -1 -1 -1 -1 -1]
Performing test
Unique pipe pIDs which are in the chunk:  [ True  True  True  True  True False]
Indices in the unique pID list:  [0 1 2 3 4]
Values of these unique pIDs:  [ 0  5  6  7 60]
pIDs in the chunk which appear in the pipe:  [False False False False False False False False False  True False False
 False False False False False False False False False False False  True
 False False  Tru

## Turn this into a function

In [207]:
def p2s_new(pipe_pIDs,chunk_pIDs,chunk_sIDs):
    """
    DESCRIPTION:
    
    Converts particle IDs to subhalo IDs. Specifically, this function:
    
    - Takes a pipe's particle IDs,
    - and takes a chunk of a simulation's particle ID and subhalo ID matchlists,
    - then outputs a list of subhalo IDs for the pipe.
    - if any pipe particle IDs appeared in the chunk,
    - the pipe's subhalo ID list will contain the corresponding value.
    - otherwise the subhalo ID will be -1 for that particle ID.
    
    Important note: Every matchlist chunk for the simulation must be looped over and 
    the results must be combined to have a comprehensive conversion for an entire pipe!
    
    NOTES:
    
    -Matchlists should have been created using Create_Particle_Subhalo_Matchlists_5.ipynb
    
    INPUTS:
    
    pipe_pIDs : [array-like] list of particle IDs in the pipe
    chunk_pIDs : [array-like] chunk of the particle-ID matchlist for a given simulation
    chunk_sIDs : [array-like] equivalent chunk of the subhalo-ID matchlist for the simulation
    
    RETURNS:
    
    pipe_sIDs : [array-like] list of subhalo IDs for the pipe, updated for any particles in
                the chunk. Any particles not in the chunk will be assigned a -1 subhalo ID.
                Every matchlist chunk for the simulation must be looped over and results
                must be combined to have a comprehensive conversion for an entire pipe!
    """
    
    #get unique particle IDs in pipe, and also inverse array and counts for each unique element
    pipe_p_uniques,pipe_p_inverse,pipe_p_counts = np.unique(pipe_pIDs,return_inverse=True,return_counts=True)
    
    #initialise a unique subhalo ID list for pipe
    pipe_s_uniques=np.ones_like(pipe_p_uniques)*-1 #initialise pipe shIDs
    
    ##find which pipe pIDs are in the chunk (NOTE: unnecessary step)
    #mask_1 = np.isin(pipe_p_uniques,chunk_pIDs)
    
    #find which chunk IDs are in the pipe
    mask_2=np.isin(chunk_pIDs,pipe_p_uniques)
    
    #create dictionary which can convert unique pIDs to their sIDs
    z = dict(zip(chunk_pIDs[mask_2],chunk_sIDs[mask_2]))
    
    #do the conversion for the uniques
    for i in range(len(pipe_p_uniques)):
        try:
            pipe_s_uniques[i]=z[pipe_p_uniques[i]]
        except:
            pass
        
    #recreate the non-uniques version of pIDs with the inverse array and counts
    #recreated_pipe_p = pipe_p_uniques[pipe_p_inverse]
    pipe_sIDs = pipe_s_uniques[pipe_p_inverse]
    
    return pipe_sIDs

In [208]:
print(p2s_new(pipe_p,chunk_p,chunk_s))

[24 27 -1 10 27 67 27 -1 38 10]


## Applying the above to data

In [210]:
#version with data

print(np.array(vals['LoSPartIDs']).shape)
print(np.array(vals['LoSPartIDs'])[0,:])

#test for only first pipe
pipe_pIDs = np.array(vals['LoSPartIDs'][0])

#initialise subhalo ID list for pipe
pipe_sIDs = np.ones_like(pipe_pIDs)*-1

#loop over each chunk of the matchlist
for j in range(len(ChunkedPartIDs)):#[298,299,300,301,302]:
    print(j)

    #load pID/shID chunk to search
    chunk_pIDs = np.load(ChunkedPartIDs[j])
    chunk_sIDs = np.load(ChunkedSubhIDs[j])
    print(chunk_pIDs.shape,chunk_sIDs.shape)
    
    #get sh ID list updated for THIS CHUNK ONLY
    #they need to be combined somehow during the loop over chunks
    out_ = p2s_new(pipe_pIDs,chunk_pIDs,chunk_sIDs)
    
    #update whole subhalo ID list with this chunk's updates
    inds = np.where(out_!=-1)
    pipe_sIDs[inds] = out_[inds]
    
    print(pipe_sIDs)
    print(np.where(pipe_sIDs!=-1))
    


(5125, 10000)
[115151180266 115151180266 115151180266 ... 146241704638 146241704638
 146241704638]
0
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
1
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
2
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
3
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
4
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
5
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
6
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
7
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
8
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
9
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
10
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),

(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
99
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
100
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
101
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
102
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
103
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
104
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
105
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
106
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
107
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
108
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64),)
109
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([], dtype=int64)

(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274, 8275, 8276, 8277,
       8278, 8280]),)
180
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274, 8275, 8276, 8277,
       8278, 8280]),)
181
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274, 8275, 8276, 8277,
       8278, 8280]),)
182
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274, 8275, 8276, 8277,
       8278, 8280]),)
183
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274, 8275, 8276, 8277,
       8278, 8280]),)
184
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274, 8275, 8276, 8277,
       8278, 8280]),)
185
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([8265, 8268, 82

(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
233
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
234
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
235
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
236
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
237
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275,

(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
281
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
282
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
283
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
284
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
285
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275,

...

(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
761
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
762
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
763
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
764
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
765
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275,

(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
809
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
810
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
811
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
812
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
813
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275,

(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
857
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
858
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
859
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
860
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
861
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275,

(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
905
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
906
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
907
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
908
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
909
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275,

(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
953
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
954
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
955
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
956
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275, 8276, 8277, 8278, 8280]),)
957
(14454722,) (14454722,)
[-1. -1. -1. ... -1. -1. -1.]
(array([2279, 2284, 2285, 8265, 8268, 8269, 8270, 8271, 8272, 8273, 8274,
       8275,

In [213]:
print(pipe_sIDs)
print(pipe_sIDs[np.where(pipe_sIDs!=-1)])

[-1. -1. -1. ... -1. -1. -1.]
[2746251. 2746251. 2746251. 1555358. 1555358. 1555358. 1555358. 1555358.
 1555358. 1555358. 1555358. 1555358. 1555358. 1555358. 1555358. 1555358.]
